In [1]:
import os
from os import listdir
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import sys
sys.path.append("../../src")
from scipy.fft import fft
from FeaturesExtractorFromTSD import create_feature_dict_from_tsd
from numpy.ma.core import negative
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, month_plot, quarter_plot
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import statistics

In [2]:
paths_to_csv = {}

# Display time series of different angles

In [3]:
path_tds_angles = '../../results/big_dataset/TSD_angles'
folders = [x[0] for x in os.walk(path_tds_angles)]
folders.remove(path_tds_angles)

In [4]:
for folder in folders:
    video_name = folder.split('\\')[-1]
    curr_paths_to_csv = [(f[:-4], os.path.join(folder, f)) for f in listdir(folder)]
    paths_to_csv[video_name] = curr_paths_to_csv

In [5]:
paths_to_csv

{'S001C': [('angl_bow_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_bow_mov.csv'),
  ('angl_head_inclination',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_head_inclination.csv'),
  ('angl_left_cheek_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_cheek_mov.csv'),
  ('angl_left_elbow_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_elbow_mov.csv'),
  ('angl_left_fingers_pos_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_fingers_pos_mov.csv'),
  ('angl_left_hips_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_hips_mov.csv'),
  ('angl_left_should_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_should_mov.csv'),
  ('angl_left_wrist_round',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_wrist_round.csv'),
  ('angl_right_cheek_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_right_cheek_mov.csv'),
  ('angl_right_elbow_mov',
   '../../results/big_dataset/TSD_angl

In [6]:
# sort values to show firstly C class then I and S
names_of_videos = [v for v in paths_to_csv.keys()]
names_of_videos.sort(key=lambda v: v[::-1])
print(names_of_videos)

['S001C', 'S002C', 'S007C', 'S008C', 'S009C', 'S002I', 'S003I', 'S005I', 'S007I', 'S009I', 'S001S', 'S002S', 'S004S', 'S007S', 'S009S']


In [7]:
angles = pd.read_csv(paths_to_csv[names_of_videos[0]][0][1])['angle'].to_list()

In [8]:
num_of_metrics = len(paths_to_csv[names_of_videos[0]])

In [9]:
paths_to_csv

{'S001C': [('angl_bow_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_bow_mov.csv'),
  ('angl_head_inclination',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_head_inclination.csv'),
  ('angl_left_cheek_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_cheek_mov.csv'),
  ('angl_left_elbow_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_elbow_mov.csv'),
  ('angl_left_fingers_pos_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_fingers_pos_mov.csv'),
  ('angl_left_hips_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_hips_mov.csv'),
  ('angl_left_should_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_should_mov.csv'),
  ('angl_left_wrist_round',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_left_wrist_round.csv'),
  ('angl_right_cheek_mov',
   '../../results/big_dataset/TSD_angles\\S001C\\angl_right_cheek_mov.csv'),
  ('angl_right_elbow_mov',
   '../../results/big_dataset/TSD_angl

In [146]:
limits_max = np.zeros(num_of_metrics)
limits_min = [10000]*num_of_metrics
for i in range(len(names_of_videos)):
    for j in range(num_of_metrics):
        print(names_of_videos[i])
        curr_metric, curr_path = paths_to_csv[names_of_videos[i]][j]
        angles = pd.read_csv(curr_path)['angle'].to_list()
        angle_max = max(angles)
        angle_min = min(angles)
        if limits_max[j] < angle_max:
            limits_max[j] = angle_max
        if limits_min[j] > angle_min:
            limits_min[j] = angle_min

S001C
S001C
S001C
S001C
S001C
S001C
S001C
S001C
S001C
S001C
S001C
S001C
S001C
S002C
S002C
S002C
S002C
S002C
S002C
S002C
S002C
S002C
S002C
S002C
S002C
S002C
S007C
S007C
S007C
S007C
S007C
S007C
S007C
S007C
S007C
S007C
S007C
S007C
S007C
S008C
S008C
S008C
S008C
S008C
S008C
S008C
S008C
S008C
S008C
S008C
S008C
S008C
S009C
S009C
S009C
S009C
S009C
S009C
S009C
S009C
S009C
S009C
S009C
S009C
S009C
S002I
S002I
S002I
S002I
S002I
S002I
S002I
S002I
S002I
S002I
S002I
S002I
S002I
S003I
S003I
S003I
S003I
S003I
S003I
S003I
S003I
S003I
S003I
S003I
S003I
S003I
S005I
S005I
S005I
S005I
S005I
S005I
S005I
S005I
S005I
S005I
S005I
S005I
S005I
S007I
S007I
S007I
S007I
S007I
S007I
S007I
S007I
S007I
S007I
S007I
S007I
S007I
S009I
S009I
S009I
S009I
S009I
S009I
S009I
S009I
S009I
S009I
S009I
S009I
S009I
S001S
S001S
S001S
S001S
S001S
S001S
S001S
S001S
S001S
S001S
S001S
S001S
S001S
S002S
S002S
S002S
S002S
S002S
S002S
S002S
S002S
S002S
S002S
S002S
S002S
S002S
S004S
S004S
S004S
S004S
S004S
S004S
S004S
S004S
S004S
S004S
S004

In [10]:
experiment_suff = "big_dataset_"

In [11]:
def smooth_time_series(time_series, window_size):
    smoothed_series = []
    for i in range(len(time_series)):
        lower_bound = max(0, i - window_size)
        upper_bound = min(len(time_series), i + window_size + 1)
        window = time_series[lower_bound:upper_bound]
        average = sum(window) / len(window)
        smoothed_series.append(average)
    return smoothed_series

In [149]:
fig, axs = plt.subplots(len(paths_to_csv['S005I']),len(names_of_videos), figsize=(260, 90))
for i in range(len(names_of_videos)):
    #for j in range(num_of_metrics):
    for j in [0,]:
        curr_metric, curr_path = paths_to_csv[names_of_videos[i]][j]
        angles = smooth_time_series(pd.read_csv(curr_path)['angle'].to_list(), 10)
        axs[j][i].plot(angles)
        axs[j][i].title.set_text(names_of_videos[i] + "\n" + curr_metric)
        axs[j][i].set_yticks(np.arange(limits_min[j], limits_max[j], step=50))
plt.tight_layout()
plt.savefig(experiment_suff+"results_consider_angles.png")
plt.close()

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x0000020EE08BF5E8> (for post_execute):


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



## Results
We can see the pattern in the following angle -> rwrist, lshoul, lhip (moving of the bow)

# Analyse the moving of the bow

In [12]:
bow_moving = []
titles = []

In [13]:
len(names_of_videos)

15

In [14]:
len(paths_to_csv)

15

In [15]:
for i in range(len(names_of_videos)):
    curr_metric, curr_path = paths_to_csv[names_of_videos[i]][0]
    print(curr_path)
    print(curr_metric)
    title = curr_path.split('/')[-1].split('\\')[1]
    titles.append(title)
    print(title)
    angles = pd.read_csv(curr_path)['angle'].to_list()
    bow_moving.append(angles)

../../results/big_dataset/TSD_angles\S001C\angl_bow_mov.csv
angl_bow_mov
S001C
../../results/big_dataset/TSD_angles\S002C\angl_bow_mov.csv
angl_bow_mov
S002C
../../results/big_dataset/TSD_angles\S007C\angl_bow_mov.csv
angl_bow_mov
S007C
../../results/big_dataset/TSD_angles\S008C\angl_bow_mov.csv
angl_bow_mov
S008C
../../results/big_dataset/TSD_angles\S009C\angl_bow_mov.csv
angl_bow_mov
S009C
../../results/big_dataset/TSD_angles\S002I\angl_bow_mov.csv
angl_bow_mov
S002I
../../results/big_dataset/TSD_angles\S003I\angl_bow_mov.csv
angl_bow_mov
S003I
../../results/big_dataset/TSD_angles\S005I\angl_bow_mov.csv
angl_bow_mov
S005I
../../results/big_dataset/TSD_angles\S007I\angl_bow_mov.csv
angl_bow_mov
S007I
../../results/big_dataset/TSD_angles\S009I\angl_bow_mov.csv
angl_bow_mov
S009I
../../results/big_dataset/TSD_angles\S001S\angl_bow_mov.csv
angl_bow_mov
S001S
../../results/big_dataset/TSD_angles\S002S\angl_bow_mov.csv
angl_bow_mov
S002S
../../results/big_dataset/TSD_angles\S004S\angl_bow_

In [16]:
print(len(bow_moving))
print(len(bow_moving[0]))

15
10760


In [17]:
def calculate_sign_change_rate(time_series):
    count = 0
    for i in range(1, len(time_series)):
        if time_series[i] * time_series[i-1] < 0:
            count += 1
    #sign_change_rate = count / (len(time_series) - 1)
    return count

def calc_area(values):
  values = np.array(values)
  indices = np.where(values > 0)[0]
  area_pos = np.trapz(values[indices])
  indices = np.where(values < 0)[0]
  area_neg = np.trapz(values[indices])
  return area_pos, area_neg

def shift_data(series):
  median = statistics.median(series)
  shifted_data = [x - median for x in series]
  return shifted_data


In [24]:

def classify_time_series(angl_bow_mov, titles):
    num_rows = 8
    num_cols = 2
    shifted_smooth_series = []
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(50, 18))
    plt.rcParams['font.size'] = 20
    axes = axes.flatten()
    sign_change_arr = []
    #print(len(angl_bow_mov))
    for i, series in enumerate(angl_bow_mov):
      smoothed_data = smooth_time_series(series, 15)
      #neg_smoothed_data = negate_below_median(smoothed_data)
      title = titles[i]
      #num_sign_change = calculate_sign_change_rate(neg_smoothed_data)
      #print(title, " ", num_sign_change)

      ax = axes[i]
        #series = eliminate_lowest_values(series,10)
        #average = sum(time_series) / len(time_series)
      median = statistics.median(smoothed_data)
      #mode = statistics.mode(smoothed_data)
      shifted_data = shift_data(smoothed_data)
      #shifted_data = smooth_time_series(shifted_data, 15)
      ax.axhline(median, color='r', linestyle='--', label='median')
      ax.axhline(0, color='g', linestyle='--', label='median')
      #smoothed_data = smooth_time_series(series, 10) #np.convolve(series, np.ones(window_size) / window_size, mode='valid')
      ax.plot(shifted_data, label='Shifted Data')
      ax.plot(smoothed_data, label='Smoothed Data')
      #ax.set_title(title)
      #ax.legend()
      #smooth_series.append(smoothed_data)
      s =calculate_sign_change_rate(shifted_data)
      a1, a2 = calc_area(shifted_data)
      #print(title, s, a1,a2, a1+a2)
      sign_change_arr.append(s)
      shifted_smooth_series.append(shifted_data)

    arr = []

    count_false = 0
    count_true = 0

    mean_len = 0
    for v in shifted_smooth_series:
        s = calculate_sign_change_rate(v)
        arr.append(s)
        mean_len+=len(v)

    thres = int(statistics.mean(arr))
    mean_len /= len(shifted_smooth_series)
    thres_norm = int(1000*thres/mean_len)

    for i, series in enumerate(shifted_smooth_series):
        s = arr[i]
        title = titles[i]
        ax = axes[i]
        s_norm = int(1000*s/len(shifted_smooth_series[i]))

        if s_norm > thres_norm:
          title += f" Shuffled - {str('S'==title[-1])}"
        else:
          a1, a2 = calc_area(series)
          if abs(a2) < a1/2:
            title += f" Inverted - {str('I'==title[-1])}"
          else:
            title += f" Correct - {str('C'==title[-1])}"

        ax.set_title(title)
        if 'False' in title:
            count_false+=1
        else:
            count_true+=1

    plt.tight_layout()
    plt.savefig(experiment_suff+"ts_classification.png")
    plt.close()
    print(f"------------------- \n true predictions: {count_true} \n false predictions: {count_false} \n")

In [25]:
classify_time_series(bow_moving, titles)

------------------- 
 true predictions: 15 
 false predictions: 0 



# Analysis of feature engineering

In [ ]:
names_of_metrics = []
for i in range(len(names_of_videos)):
    for j in range(num_of_metrics):
        curr_metric, curr_path = paths_to_csv[names_of_videos[i]][j]
        names_of_metrics.append(curr_metric)
    break

sample_tsd = pd.read_csv(curr_path)['angle'].to_list()
names_metrics_and_features = []
for metr in names_of_metrics:
    for feature_n in create_feature_dict_from_tsd(sample_tsd).keys():
        names_metrics_and_features.append(f"{metr}_{feature_n}")

In [ ]:
di_df = {}
di_df["features_of_metrics"] = names_metrics_and_features

In [ ]:
for i in range(len(names_of_videos)):
    curr_metrics = []
    for j in range(num_of_metrics):
        curr_metric, curr_path = paths_to_csv[names_of_videos[i]][j]
        angles_tsd = pd.read_csv(curr_path)['angle'].to_list()
        features_di = create_feature_dict_from_tsd(angles_tsd)
        features = list(features_di.values())
        curr_metrics.extend(features)
    di_df[names_of_videos[i]] = curr_metrics

In [ ]:
df = pd.DataFrame(data=di_df)
df.to_csv("../../results/TSD_angles/extracted_features.csv")

In [ ]:
df

In [ ]:
correl = df.corr()

In [ ]:
correl.style.highlight_min(color = 'gray', axis = 1)

In [ ]:
correl_2 = df.drop('S002C', axis=1).corr()

In [ ]:
correl_2.style.highlight_min(color = 'gray', axis = 1)

In [32]:
correl_3 = df.drop('S002C', axis=1).drop('S005I', axis=1).corr()

In [33]:
correl_3.style.highlight_min(color = 'gray', axis = 1)

,S001C,S002I,S003I,S001S,S004S,S009S
S001C,1.000000,0.975323,0.984160,0.977015,0.997006,0.986382
S002I,0.975323,1.000000,0.994115,0.982389,0.982174,0.977898
S003I,0.984160,0.994115,1.000000,0.991034,0.990676,0.986786
S001S,0.977015,0.982389,0.991034,1.000000,0.987413,0.982940
S004S,0.997006,0.982174,0.990676,0.987413,1.000000,0.986990
S009S,0.986382,0.977898,0.986786,0.982940,0.986990,1.000000
